In [1]:
from test_utils import *
from swayam import Agent

## JSON Output

Beyond basic conversational usage of an LLM, especially when you want to integrate the output into a subsequent local functionality, getting structured output can be of great help.

In [2]:
agent = Agent()

In [3]:
## Example from OpenAI website

from swayam import Tool

builder = Tool.structure_builder("CalendarEvent")
builder.add_field("name", type=str, desc="Name of the event")
builder.add_field("date", type=str, desc="Date on which the event takes place.")
builder.add_field("participants", type=list[str], desc="Names of participants.")
builder.add_field("city", type=list[str], desc="City where the event takes place.")

CalendarEvent = builder.build()
    


In [4]:
print(CalendarEvent.schema_json())

{"properties": {"name": {"description": "Name of the event", "title": "Name", "type": "string"}, "date": {"description": "Date on which the event takes place.", "title": "Date", "type": "string"}, "participants": {"description": "Names of participants.", "items": {"type": "string"}, "title": "Participants", "type": "array"}, "city": {"description": "City where the event takes place.", "items": {"type": "string"}, "title": "City", "type": "array"}}, "required": ["name", "date", "participants", "city"], "title": "CalendarEvent", "type": "object"}


In [5]:
prompt = "Extract event information: Alice and Bob are going to a science fair on Friday."
agent.execute(prompt, response_format=CalendarEvent, reset_report=True)

ParsedChatCompletionMessage[CalendarEvent](content='{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"],"city":[] }', refusal=None, role='assistant', function_call=None, tool_calls=[], parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'], city=[]))

Execution of multiple queries will need each one to be context free from another else there is an OpenAI exception relating to a tool call missing.

In [6]:
prompts = [
    "Extract event information: Alice and Bob are going to a science fair on Friday.",
    "Extract event information: We are going to a science fair on Tuesday.",
    "Extract event information: Rahul and Pepe are going to the party on 12th August."
]
agent.execute(*prompts, response_format=CalendarEvent, reset_report=True, same_context=False)

[ParsedChatCompletionMessage[CalendarEvent](content='{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"],"city":[] }', refusal=None, role='assistant', function_call=None, tool_calls=[], parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'], city=[])),
 ParsedChatCompletionMessage[CalendarEvent](content='{"name":"Science Fair","date":"Tuesday","participants":[],"city":[]}', refusal=None, role='assistant', function_call=None, tool_calls=[], parsed=CalendarEvent(name='Science Fair', date='Tuesday', participants=[], city=[])),
 ParsedChatCompletionMessage[CalendarEvent](content='{"name":"Party","date":"2023-08-12","participants":["Rahul","Pepe"],"city":[] }', refusal=None, role='assistant', function_call=None, tool_calls=[], parsed=CalendarEvent(name='Party', date='2023-08-12', participants=['Rahul', 'Pepe'], city=[]))]